# 🤝 AgentTool - Run Any Agent

The **AgentTool** is an OpenSearch ML Commons tool that enables an agent to run (call) any other registered agent. This is the foundation for building agent composition patterns and multi-agent systems.

## 📚 Learning Objectives

In this notebook, you'll learn:
1. ✅ How to register an agent for AgentTool to call
2. ✅ How to register a flow agent using AgentTool
3. ✅ How to execute an agent via AgentTool
4. ✅ Building multi-agent systems with agent delegation
5. ✅ Best practices for agent composition

---

## 🎯 What is AgentTool?

**AgentTool** (introduced in OpenSearch 2.13) is a specialized tool that allows agents to call other agents. This enables:
- 🏗️ **Agent Reuse**: Any registered agent can be called by other agents
- 🤝 **Agent Composition**: Build complex workflows from simple agents
- 🎯 **Task Delegation**: Agents can delegate to specialized agents
- 📊 **Modular Design**: Break systems into independent, composable agents

**How it Works**:
```
User Query
   ↓
Main Agent (with AgentTool)
   ↓
AgentTool calls Sub-Agent
   ↓
Sub-Agent processes and returns result
   ↓
Result back to Main Agent (or User)
```

---

## Step 1: Set Up an Agent for AgentTool to Run

First, we need to register and deploy a base agent that will be called by AgentTool. According to the OpenSearch documentation, this agent should perform actual work (typically using MLModelTool).

Following the documentation pattern, we'll:
1. Set up an OpenAI connector and model
2. Register a flow agent using MLModelTool (this is what AgentTool will call)
3. Register another agent with AgentTool pointing to the first agent

In [1]:
import sys
import json
import time

sys.path.append('..')
from agent_helpers import (
    get_os_client,
    configure_cluster_for_openai,
    create_embedding_model,
    create_ingest_pipeline_and_index,
    create_openai_connector,
    register_and_deploy_openai_model,
    create_flow_agent,
    execute_agent,
    cleanup_resources
)

print("✅ Libraries imported successfully!")

✅ Libraries imported successfully!


## Step 2: Initialize OpenSearch Client and Set Up Base Agent

In [2]:
client = get_os_client()
info = client.info()
print(f"✅ Connected to OpenSearch: {info['cluster_name']}")
print(f"📊 Version: {info['version']['number']}")

bulk_body = [
    {"index": {"_id": "1"}},
    {"text": "Chart and table of population level and growth rate for the Ogden-Layton metro area from 1950 to 2023. United Nations population projections are also included through the year 2035.\nThe current metro area population of Ogden-Layton in 2023 is 750,000, a 1.63% increase from 2022.\nThe metro area population of Ogden-Layton in 2022 was 738,000, a 1.79% increase from 2021.\nThe metro area population of Ogden-Layton in 2021 was 725,000, a 1.97% increase from 2020.\nThe metro area population of Ogden-Layton in 2020 was 711,000, a 2.16% increase from 2019."},
    {"index": {"_id": "2"}},
    {"text": "Chart and table of population level and growth rate for the New York City metro area from 1950 to 2023. United Nations population projections are also included through the year 2035.\nThe current metro area population of New York City in 2023 is 18,937,000, a 0.37% increase from 2022.\nThe metro area population of New York City in 2022 was 18,867,000, a 0.23% increase from 2021.\nThe metro area population of New York City in 2021 was 18,823,000, a 0.1% increase from 2020.\nThe metro area population of New York City in 2020 was 18,804,000, a 0.01% decline from 2019."},
    {"index": {"_id": "3"}},
    {"text": "Chart and table of population level and growth rate for the Chicago metro area from 1950 to 2023. United Nations population projections are also included through the year 2035.\nThe current metro area population of Chicago in 2023 is 8,937,000, a 0.4% increase from 2022.\nThe metro area population of Chicago in 2022 was 8,901,000, a 0.27% increase from 2021.\nThe metro area population of Chicago in 2021 was 8,877,000, a 0.14% increase from 2020.\nThe metro area population of Chicago in 2020 was 8,865,000, a 0.03% increase from 2019."},
    {"index": {"_id": "4"}},
    {"text": "Chart and table of population level and growth rate for the Miami metro area from 1950 to 2023. United Nations population projections are also included through the year 2035.\nThe current metro area population of Miami in 2023 is 6,265,000, a 0.8% increase from 2022.\nThe metro area population of Miami in 2022 was 6,215,000, a 0.78% increase from 2021.\nThe metro area population of Miami in 2021 was 6,167,000, a 0.74% increase from 2020.\nThe metro area population of Miami in 2020 was 6,122,000, a 0.71% increase from 2019."},
    {"index": {"_id": "5"}},
    {"text": "Chart and table of population level and growth rate for the Austin metro area from 1950 to 2023. United Nations population projections are also included through the year 2035.\nThe current metro area population of Austin in 2023 is 2,228,000, a 2.39% increase from 2022.\nThe metro area population of Austin in 2022 was 2,176,000, a 2.79% increase from 2021.\nThe metro area population of Austin in 2021 was 2,117,000, a 3.12% increase from 2020.\nThe metro area population of Austin in 2020 was 2,053,000, a 3.43% increase from 2019."},
    {"index": {"_id": "6"}},
    {"text": "Chart and table of population level and growth rate for the Seattle metro area from 1950 to 2023. United Nations population projections are also included through the year 2035.\nThe current metro area population of Seattle in 2023 is 3,519,000, a 0.86% increase from 2022.\nThe metro area population of Seattle in 2022 was 3,489,000, a 0.81% increase from 2021.\nThe metro area population of Seattle in 2021 was 3,461,000, a 0.82% increase from 2020.\nThe metro area population of Seattle in 2020 was 3,433,000, a 0.79% increase from 2019."}
]

# Configure cluster and create OpenAI model
configure_cluster_for_openai(client)
connector_id = create_openai_connector(client)
embedding_model_id = create_embedding_model(client)
index_name = create_ingest_pipeline_and_index(client, embedding_model_id, bulk_body)
model_id = register_and_deploy_openai_model(client, connector_id)
print(f"✅ OpenAI model ready: {model_id}")
print(f"✅ Index name: {index_name}")

✅ Connected to OpenSearch: docker-cluster
📊 Version: 3.3.0
   Configuring cluster settings for OpenAI connector...
   ✓ Cluster settings configured successfully
   Creating OpenAI connector for gpt-4o-mini...
   ✓ Connector created: y1tNiZsBLQ1mV2UNUSjn
STEP 2: Setting up Embedding Model (Semantic Search)...
   Registering HuggingFace sentence-transformers model...
   Task ID: zFtNiZsBLQ1mV2UNUSj7
   ⏳ Waiting for embedding model registration to complete...
      Registration status: CREATED
      Registration status: COMPLETED
   ✓ Embedding model registered with ID: zVtNiZsBLQ1mV2UNVSi0

   Deploying embedding model...
   ⏳ Waiting for embedding model deployment...
      Model status: DEPLOYING
      Model status: DEPLOYED
      ✓ Model deployed successfully!
STEP 3: Creating Ingest Pipeline and Index...
   ✓ Ingest pipeline created with ID: openai-agent-pipeline_1767535119
   Creating index with vector field for semantic search...
   ✓ Index created with name: openai_agent_data_1767

## Step 3: Register the Base Agent (Using MLModelTool)

This is the agent that AgentTool will call. It performs actual work using MLModelTool.
According to the docs, this is the critical first step - we need a functioning agent for AgentTool to invoke.

In [3]:
# Create the base agent that will be called by AgentTool
# This agent uses MLModelTool to answer questions
base_agent_tools = [{
                "type": "VectorDBTool",
                "parameters": {
                    "model_id": embedding_model_id,
                    "index": index_name,
                    "embedding_field": "embedding",
                    "source_field": [
                        "text"
                    ],
                    "input": "${parameters.question}"
                }
            },
                        {
                "type": "MLModelTool",
                "description": "A tool using OpenAI GPT to answer questions based on context",
                "parameters": {
                    "model_id": model_id,
                    "messages": "[{\"role\": \"system\", \"content\": \"You are a professional data analyst. You will always answer a question based on the given context first. If the answer is not directly shown in the context, you will analyze the data and find the answer. If you don't know the answer, just say you don't know.\"}, {\"role\": \"user\", \"content\": \"Context:\\n${parameters.VectorDBTool.output}\\n\\nQuestion: ${parameters.question}\"}]"
                }
            }]

# Register the base agent with a clear descriptive name
base_agent_id = create_flow_agent(
    client, 
    "Base_Data_Analyst",
    "Base agent that answers data analysis questions",
    base_agent_tools
)
print(f"✅ Base agent registered: {base_agent_id}")
print(f"   This is the agent that AgentTool will call")

# Test agent
print(f"✅ Testing base agent directly...")
print("="*70)

test_params = {
    "question": "What's the population increase of Seattle from 2021 to 2023"
}

try:
    test_response = execute_agent(client, base_agent_id, test_params)
    print("✅ Base agent response:")
    print(json.dumps(test_response, indent=2))
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()


   Registering flow agent: Base_Data_Analyst...
   ✓ Agent registered: 21tOiZsBLQ1mV2UNvyhS
✅ Base agent registered: 21tOiZsBLQ1mV2UNvyhS
   This is the agent that AgentTool will call
✅ Testing base agent directly...
✅ Base agent response:
{
  "inference_results": [
    {
      "output": [
        {
          "name": "MLModelTool",
          "result": "{\"id\":\"chatcmpl-CuIwBMdjiM0gKUB5tSmtFXhTU6hRY\",\"object\":\"chat.completion\",\"created\":1.767535199E9,\"model\":\"gpt-4o-mini-2024-07-18\",\"choices\":[{\"index\":0.0,\"message\":{\"role\":\"assistant\",\"content\":\"The population of Seattle in 2021 was 3,461,000, and in 2023 it is 3,519,000. \\n\\nTo find the population increase from 2021 to 2023:\\n3,519,000 - 3,461,000 = 58,000\\n\\nThe population increase of Seattle from 2021 to 2023 is 58,000.\",\"refusal\":null,\"annotations\":[]},\"logprobs\":null,\"finish_reason\":\"stop\"}],\"usage\":{\"prompt_tokens\":498.0,\"completion_tokens\":82.0,\"total_tokens\":580.0,\"prompt_token

## Step 4: Register a Flow Agent Using AgentTool

Now we register a flow agent that uses AgentTool to call the base agent.
According to the OpenSearch documentation, AgentTool requires:
- **agent_id**: The ID of the agent to run (required)
- When executed, it accepts **question** parameter (required)

In [4]:
# Register a flow agent that uses AgentTool to call the base agent
# This follows the pattern from the OpenSearch documentation exactly
# Using direct API call instead of helper method for transparency

agent_tool_body = {
    "name": "Test agent tool",
    "type": "flow",
    "description": "this is a test agent",
    "tools": [
        {
            "type": "AgentTool",
            "description": "A general agent to answer any question",
            "parameters": {
                "agent_id": base_agent_id
            }
        }
    ]
}

print("📝 Registering orchestrator agent via direct API call...")
print("="*70)

try:
    response = client.transport.perform_request(
        'POST', 
        '/_plugins/_ml/agents/_register',
        body=agent_tool_body
    )
    orchestrator_agent_id = response.get('agent_id')
    print(f"✅ Orchestrator agent registered: {orchestrator_agent_id}")
    print(f"   Name: Test agent tool")
    print(f"   Type: flow")
    print(f"   This agent uses AgentTool to call: {base_agent_id}")
except Exception as e:
    print(f"❌ Error registering orchestrator agent: {e}")
    import traceback
    traceback.print_exc()

📝 Registering orchestrator agent via direct API call...
✅ Orchestrator agent registered: 3FtPiZsBLQ1mV2UNoijh
   Name: Test agent tool
   Type: flow
   This agent uses AgentTool to call: 21tOiZsBLQ1mV2UNvyhS


## Step 5: Execute the Agent via AgentTool

Now we execute the orchestrator agent, which will use AgentTool to call the base agent.
According to the documentation, AgentTool accepts a "question" parameter.

In [5]:
# Execute the orchestrator agent with a question
# According to OpenSearch docs, AgentTool accepts 'question' parameter

parameters = {
    "question": "What's the population increase of Seattle from 2021 to 2023"
}

print("❓ Executing orchestrator agent via AgentTool...")
print("="*70)
print(f"Question: {parameters['question']}")
print("="*70)

try:
    response = execute_agent(client, orchestrator_agent_id, parameters)
    print("\n✅ Response from orchestrator (via AgentTool):")
    print(json.dumps(response, indent=2))
except Exception as e:
    error_str = str(e)
    print(f"❌ Error: {e}")
    print(f"\n💡 Troubleshooting for 400 IllegalArgumentException:")
    print(f"   • The model may not be fully deployed yet. Try cell 5b diagnostic.")
    print(f"   • AgentTool forwards the 'question' parameter to the base agent.")
    print(f"   • Run cell 5b to verify agent setup.")
    print(f"   • Run cell 5c Test 1 to verify base agent works independently.")
    print(f"\n📋 Full traceback:")
    import traceback
    traceback.print_exc()

❓ Executing orchestrator agent via AgentTool...
Question: What's the population increase of Seattle from 2021 to 2023

✅ Response from orchestrator (via AgentTool):
{
  "inference_results": [
    {
      "output": [
        {
          "name": "MLModelTool",
          "result": "{\"id\":\"chatcmpl-CuIxZXRBDGPGN6bWqGCrTe0207oEy\",\"object\":\"chat.completion\",\"created\":1.767535285E9,\"model\":\"gpt-4o-mini-2024-07-18\",\"choices\":[{\"index\":0.0,\"message\":{\"role\":\"assistant\",\"content\":\"The population of Seattle in 2021 was 3,461,000 and in 2023 it is 3,519,000. \\n\\nTo find the population increase from 2021 to 2023:\\n3,519,000 - 3,461,000 = 58,000\\n\\nThe population increase of Seattle from 2021 to 2023 is 58,000.\",\"refusal\":null,\"annotations\":[]},\"logprobs\":null,\"finish_reason\":\"stop\"}],\"usage\":{\"prompt_tokens\":498.0,\"completion_tokens\":81.0,\"total_tokens\":579.0,\"prompt_tokens_details\":{\"cached_tokens\":0.0,\"audio_tokens\":0.0},\"completion_tokens

## Step 5a: Sanity Test with Examples

Run sanity checks to verify the orchestrator agent is working correctly via AgentTool.

In [6]:
# Sanity checks with example tests
print("🧪 SANITY TEST: Verify AgentTool Functionality")
print("="*70)

test_cases = [
    {
        "name": "Simple Question",
        "question": "What is machine learning?"
    },
    {
        "name": "Technical Question",
        "question": "How does a neural network work?"
    },
    {
        "name": "Domain-Specific Question",
        "question": "What is the difference between supervised and unsupervised learning?"
    }
]

results = []
for i, test_case in enumerate(test_cases, 1):
    print(f"\n✅ Test {i}: {test_case['name']}")
    print(f"   Question: {test_case['question']}")
    print(f"   Executing via AgentTool...")
    
    try:
        response = execute_agent(client, orchestrator_agent_id, {"question": test_case['question']})
        
        # Sanity checks
        has_results = 'inference_results' in response
        has_output = False
        output_text = ""
        
        if has_results and len(response['inference_results']) > 0:
            first_result = response['inference_results'][0]
            if 'output' in first_result and len(first_result['output']) > 0:
                has_output = True
                output_text = first_result['output'][0].get('result', '')
        
        # Print results
        if has_output and len(output_text) > 0:
            print(f"   ✅ PASS - Got response ({len(output_text)} chars)")
            print(f"      Preview: {output_text[:100]}...")
            results.append((test_case['name'], "PASS"))
        else:
            print(f"   ⚠️  PARTIAL - Response structure exists but no output")
            print(f"      Response: {json.dumps(response, indent=2)[:200]}...")
            results.append((test_case['name'], "PARTIAL"))
            
    except Exception as e:
        print(f"   ❌ FAIL - {str(e)[:100]}")
        results.append((test_case['name'], "FAIL"))

# Summary
print("\n" + "="*70)
print("📊 SANITY TEST SUMMARY")
print("="*70)
for test_name, result in results:
    status_icon = "✅" if result == "PASS" else "⚠️ " if result == "PARTIAL" else "❌"
    print(f"{status_icon} {test_name}: {result}")

passed = sum(1 for _, r in results if r == "PASS")
total = len(results)
print(f"\n📈 Result: {passed}/{total} tests passed")

if passed == total:
    print("✅ All sanity tests passed! AgentTool is working correctly.")
elif passed > 0:
    print("⚠️  Some tests passed. AgentTool is partially functional.")
else:
    print("❌ All tests failed. Check agent configuration and model deployment.")

🧪 SANITY TEST: Verify AgentTool Functionality

✅ Test 1: Simple Question
   Question: What is machine learning?
   Executing via AgentTool...
   ✅ PASS - Got response (627 chars)
      Preview: {"id":"chatcmpl-CuIy2IKMFkjtMS4YPu5VTkj4ZQ8hS","object":"chat.completion","created":1.767535314E9,"m...

✅ Test 2: Technical Question
   Question: How does a neural network work?
   Executing via AgentTool...
   ✅ PASS - Got response (627 chars)
      Preview: {"id":"chatcmpl-CuIy2Ftzi5zxXWKICZkdPX7V23oJh","object":"chat.completion","created":1.767535314E9,"m...

✅ Test 3: Domain-Specific Question
   Question: What is the difference between supervised and unsupervised learning?
   Executing via AgentTool...
   ✅ PASS - Got response (627 chars)
      Preview: {"id":"chatcmpl-CuIy34nfdgX8U83xTPGsbdahhd0R2","object":"chat.completion","created":1.767535315E9,"m...

📊 SANITY TEST SUMMARY
✅ Simple Question: PASS
✅ Technical Question: PASS
✅ Domain-Specific Question: PASS

📈 Result: 3/3 tests passed
✅ 

## Step 6: Test Direct Agent Call (Comparison)

Let's also test calling the base agent directly to compare behavior.

In [7]:
# Test calling the base agent directly
print("📊 Calling base agent DIRECTLY (no AgentTool)...")
print("="*70)

parameters_base = {
    "question": "What are the economic indicators for Q3 2025?"
}

try:
    response_base = execute_agent(client, base_agent_id, parameters_base)
    print("✅ Response from base agent (direct call):")
    print(json.dumps(response_base, indent=2))
except Exception as e:
    print(f"❌ Error: {e}")

📊 Calling base agent DIRECTLY (no AgentTool)...
✅ Response from base agent (direct call):
{
  "inference_results": [
    {
      "output": [
        {
          "name": "MLModelTool",
          "result": "{\"id\":\"chatcmpl-CuIyMbPdQCkqqoVyjf1ZdWGNchBXD\",\"object\":\"chat.completion\",\"created\":1.767535334E9,\"model\":\"gpt-4o-mini-2024-07-18\",\"choices\":[{\"index\":0.0,\"message\":{\"role\":\"assistant\",\"content\":\"I don't know.\",\"refusal\":null,\"annotations\":[]},\"logprobs\":null,\"finish_reason\":\"stop\"}],\"usage\":{\"prompt_tokens\":496.0,\"completion_tokens\":4.0,\"total_tokens\":500.0,\"prompt_tokens_details\":{\"cached_tokens\":0.0,\"audio_tokens\":0.0},\"completion_tokens_details\":{\"reasoning_tokens\":0.0,\"audio_tokens\":0.0,\"accepted_prediction_tokens\":0.0,\"rejected_prediction_tokens\":0.0}},\"service_tier\":\"default\",\"system_fingerprint\":\"fp_c4585b5b9c\"}"
        }
      ]
    }
  ]
}


## Step 7: Multi-Agent Example - Agent Composition

Demonstrate how AgentTool enables building complex multi-agent systems where agents call other agents.
This follows the agent composition pattern where specialized agents delegate to other specialized agents.

In [8]:
# Create a specialized math agent (another base agent)
math_agent_tools = [{
    "type": "MLModelTool",
    "parameters": {
        "model_id": model_id,
        "messages": "[{\"role\": \"system\", \"content\": \"You are a professional data analyst. You will always answer a question based on the given context first. If the answer is not directly shown in the context, you will analyze the data and find the answer. If you don't know the answer, just say you don't know.\"}, {\"role\": \"user\", \"content\": \"Context:\\nQuestion: ${parameters.question}\"}]"
    }
}]


math_agent_id = create_flow_agent(
    client,
    "Specialized_Math_Agent",
    "Specialized agent for mathematical problem solving",
    math_agent_tools
)
print(f"✅ Specialized math agent created: {math_agent_id}")

# Now create an orchestrator agent that calls the math agent via AgentTool
math_orchestrator_tools = [{
    "type": "AgentTool",
    "description": "Delegates math problems to the math specialist agent",
    "parameters": {
        "agent_id": math_agent_id
    }
}]

math_orchestrator_id = create_flow_agent(
    client,
    "Math_Orchestrator",
    "Orchestrator that delegates math problems via AgentTool",
    math_orchestrator_tools
)
print(f"✅ Math orchestrator created: {math_orchestrator_id}")

print("\n💡 Agent composition pattern:")
print(f"   User Query → Math Orchestrator → AgentTool → Math Agent → Response")

   Registering flow agent: Specialized_Math_Agent...
   ✓ Agent registered: 3VtSiZsBLQ1mV2UNCyjN
✅ Specialized math agent created: 3VtSiZsBLQ1mV2UNCyjN
   Registering flow agent: Math_Orchestrator...
   ✓ Agent registered: 3ltSiZsBLQ1mV2UNCyjd
✅ Math orchestrator created: 3ltSiZsBLQ1mV2UNCyjd

💡 Agent composition pattern:
   User Query → Math Orchestrator → AgentTool → Math Agent → Response


## Step 8: Execute Specialized Agent via AgentTool

In [9]:
# Test the math orchestrator agent via AgentTool
parameters = {
    "question": "If a company's revenue grew from $2M to $2.5M, what's the percentage increase?"
}

print("🧮 Executing math orchestrator agent via AgentTool...")
print("="*70)
print(f"Question: {parameters['question']}")
print("="*70)

try:
    math_response = execute_agent(client, math_orchestrator_id, parameters)
    print("\n✅ Response from math orchestrator (via AgentTool):")
    print(json.dumps(math_response, indent=2))
    print(math_response)
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()

🧮 Executing math orchestrator agent via AgentTool...
Question: If a company's revenue grew from $2M to $2.5M, what's the percentage increase?

✅ Response from math orchestrator (via AgentTool):
{
  "inference_results": [
    {
      "output": [
        {
          "name": "response",
          "result": "{\"id\":\"chatcmpl-CuIztjkvCsKeh2IXzmiZyELqxZZie\",\"object\":\"chat.completion\",\"created\":1.767535429E9,\"model\":\"gpt-4o-mini-2024-07-18\",\"choices\":[{\"index\":0.0,\"message\":{\"role\":\"assistant\",\"content\":\"To calculate the percentage increase in revenue, you can use the formula:\\n\\n\\\\[\\n\\\\text{Percentage Increase} = \\\\left( \\\\frac{\\\\text{New Value} - \\\\text{Old Value}}{\\\\text{Old Value}} \\\\right) \\\\times 100\\n\\\\]\\n\\nIn this case, the old value is $2M and the new value is $2.5M.\\n\\n\\\\[\\n\\\\text{Percentage Increase} = \\\\left( \\\\frac{2.5M - 2M}{2M} \\\\right) \\\\times 100 = \\\\left( \\\\frac{0.5M}{2M} \\\\right) \\\\times 100 = 0.25 \

## Below has latex formatted math formulae

In [10]:
from IPython.display import Markdown, display

# Extract and display the response in human-readable format
result_content = json.loads(math_response['inference_results'][0]['output'][0]['result'])['choices'][0]['message']['content']
display(Markdown(result_content))

To calculate the percentage increase in revenue, you can use the formula:

\[
\text{Percentage Increase} = \left( \frac{\text{New Value} - \text{Old Value}}{\text{Old Value}} \right) \times 100
\]

In this case, the old value is $2M and the new value is $2.5M.

\[
\text{Percentage Increase} = \left( \frac{2.5M - 2M}{2M} \right) \times 100 = \left( \frac{0.5M}{2M} \right) \times 100 = 0.25 \times 100 = 25\%
\]

So, the percentage increase in the company's revenue is 25%.

## 🎓 Key Takeaways - AgentTool Fundamentals

### What is AgentTool?
AgentTool (introduced in OpenSearch 2.13) allows agents to call other agents, enabling:
- 🤝 **Agent Reuse**: Any registered agent can be called
- 🏗️ **Modular Architecture**: Build agents from other agents
- 📊 **Agent Composition**: Create hierarchical agent systems
- 🎯 **Task Delegation**: Specialize agents and delegate tasks

### The Three-Step Pattern (From Documentation)

**Step 1: Set up an agent for AgentTool to run**
- Create an agent with MLModelTool that performs actual work
- This is the "base agent" that will be invoked

**Step 2: Register a flow agent that will run the AgentTool**
- Create an orchestrator agent with AgentTool tool
- Reference the base agent via agent_id parameter

**Step 3: Run the agent**
- Execute the orchestrator agent with parameters
- Pass "question" parameter for LLM interaction

### Code Pattern
```python
# Step 1: Create base agent with MLModelTool
base_agent_tools = [{
    "type": "MLModelTool",
    "parameters": {"model_id": model_id, "prompt": "..."}
}]
base_agent_id = create_flow_agent(client, "Base_Agent", "...", base_agent_tools)

# Step 2: Create orchestrator with AgentTool
orchestrator_tools = [{
    "type": "AgentTool",
    "parameters": {"agent_id": base_agent_id}
}]
orch_id = create_flow_agent(client, "Orchestrator", "...", orchestrator_tools)

# Step 3: Execute with question parameter
result = execute_agent(client, orch_id, {"question": "..."})
```

### AgentTool Parameters

| Parameter | Type | Required | Description |
|-----------|------|----------|-------------|
| agent_id | String | Yes (Register) | The agent ID of the agent to run |
| question | String | Yes (Execute) | The natural language question for the LLM |

In [11]:
# Print summary of all agents created
print("\n" + "="*70)
print("📊 SUMMARY OF AGENTS CREATED")
print("="*70)

agents_summary = [
    ("Base Data Analyst", base_agent_id, "Uses MLModelTool for data analysis"),
    ("Test Orchestrator", orchestrator_agent_id, "Uses AgentTool to call Base Data Analyst"),
    ("Specialized Math Agent", math_agent_id, "Uses MLModelTool for math problems"),
    ("Math Orchestrator", math_orchestrator_id, "Uses AgentTool to call Math Agent"),
]

for idx, (name, agent_id, description) in enumerate(agents_summary, 1):
    print(f"\n{idx}. {name}")
    print(f"   ID: {agent_id}")
    print(f"   Purpose: {description}")

print("\n" + "="*70)
print("💡 Key Insight: AgentTool Pattern")
print("="*70)
print("""
AgentTool enables a 'Composition' pattern:

Base Agents (with MLModelTool)
    ↓
Orchestrator Agents (with AgentTool)
    ↓
Higher-level Orchestrators (optional, with more AgentTools)

This allows building complex multi-agent systems through composition.
Each agent focuses on one task, and AgentTool coordinates them.
""")


📊 SUMMARY OF AGENTS CREATED

1. Base Data Analyst
   ID: 21tOiZsBLQ1mV2UNvyhS
   Purpose: Uses MLModelTool for data analysis

2. Test Orchestrator
   ID: 3FtPiZsBLQ1mV2UNoijh
   Purpose: Uses AgentTool to call Base Data Analyst

3. Specialized Math Agent
   ID: 3VtSiZsBLQ1mV2UNCyjN
   Purpose: Uses MLModelTool for math problems

4. Math Orchestrator
   ID: 3ltSiZsBLQ1mV2UNCyjd
   Purpose: Uses AgentTool to call Math Agent

💡 Key Insight: AgentTool Pattern

AgentTool enables a 'Composition' pattern:

Base Agents (with MLModelTool)
    ↓
Orchestrator Agents (with AgentTool)
    ↓
Higher-level Orchestrators (optional, with more AgentTools)

This allows building complex multi-agent systems through composition.
Each agent focuses on one task, and AgentTool coordinates them.



## 🎓 Best Practices and Patterns

### Architecture Patterns with AgentTool

1. **Simple Delegation**
   ```
   Orchestrator (AgentTool) → Base Agent → Result
   ```

2. **Multi-Agent Specialization**
   ```
   Orchestrator A (AgentTool) → Specialist Agent A
   Orchestrator B (AgentTool) → Specialist Agent B
   Orchestrator C (AgentTool) → Specialist Agent C
   ```

3. **Hierarchical Delegation**
   ```
   Coordinator (AgentTool calls Manager)
   → Manager (AgentTool calls Worker)
   → Worker (MLModelTool does work)
   ```

### Important Constraints and Notes

- ✅ **Each agent can have ONE AgentTool** (not multiple)
- ✅ **AgentTool forwards parameters** to the called agent
- ✅ **Required parameter**: "question" when executing
- ✅ **Use agent_id** to reference the agent to call
- ✅ **Flow agents run tools in sequence** and return last tool's output

### Use Cases

| Use Case | Pattern |
|----------|---------|
| Task delegation | Orchestrator → Specialist Agent |
| Data pipeline | Sequential orchestrators calling each other |
| Domain expertise | Each agent masters one domain |
| Modular design | Reusable agents composed together |

### Testing Individual Tools

According to OpenSearch docs, you can also test individual tools using the **Execute Tool API** for standalone operations.

## 🧹 Cleanup (Optional)

In [ ]:
# Optional: Cleanup all agents and resources
# Uncomment to run cleanup

# cleanup_resources(
#     client=client,
#     agent_ids=[
#         orchestrator_agent_id,
#         math_orchestrator_id,
#         base_agent_id,
#         math_agent_id
#     ],
#     model_ids=[model_id],
#     connector_ids=[connector_id]
# )
# print("✅ Cleanup complete!")

print("\n✅ Notebook execution completed!")

## ? References and Next Steps

### OpenSearch Documentation
- [AgentTool Documentation](https://docs.opensearch.org/latest/ml-commons-plugin/agents-tools/tools/agent-tool/)
- [MLModelTool Documentation](https://docs.opensearch.org/latest/ml-commons-plugin/agents-tools/tools/ml-model-tool/)
- [Execute Tool API](https://docs.opensearch.org/latest/ml-commons-plugin/api/execute-tool/)
- [ML Commons Agents and Tools](https://docs.opensearch.org/latest/ml-commons-plugin/agents-tools/)

### Next Topics to Explore
- **WebSearchTool**: Search the web and return results
- **ScratchpadTool**: Add memory to multi-agent systems  
- **RAGTool**: Combine vector search with agent composition
- **Multi-hop Agents**: Chain multiple tool calls together
- **Error Handling**: Implement fallbacks and retry logic

### Key Takeaway
AgentTool enables building sophisticated multi-agent systems by allowing agents to compose other agents, creating modular, reusable, and scalable AI systems.